## Docking molecular

Una vez obtenidas todas las estructuras enn formato .pdbqt, es posible realizar un estudio de acoplamiento molecular Docking utilizando AutodockVina.

In [5]:
# DOCKING MOLECULAR
# Encontrando coordenadas para la gridbox

import subprocess
import MDAnalysis as mda
import csv
from pathlib import Path

# --- DOCKING MOLECULAR ---
# Definir la ruta a AutoDock Vina
vina_path = r"C:\Users\User\Downloads\python\AutodockVina\vina.exe"
subprocess.run([vina_path, "--help"], check=True)

# Centro de gridbox
original_structure = mda.Universe("protein_structures/5I6X.pdb")
ligand_mda = original_structure.select_atoms("resname 8PR")
pocket_center = ligand_mda.center_of_geometry()
print("Centro de la caja:", pocket_center)

# Tamaño de gridbox (actualmente no está calculado correctamente, pero se deja como estaba)
ligand_box = ligand_mda.positions.max(axis=0) - ligand_mda.positions.max(axis=0) + 5  
ligand_box

# Convertir a listas
pocket_center = pocket_center.tolist()
ligand_box = ligand_box.tolist()

# Parámetros
pdb_id = "5I6X"
ligand = "8PR"
receptor_file = Path("C:/Users/User/Downloads/python/receptores/5I6X.pdbqt")  # ⚠️ Asegúrate de que exista

# Directorios
ligands_dir = Path("C:/Users/User/Downloads/python/ligands_pdbqt")
results_dir = Path("C:/Users/User/Downloads/python/docking_results")
csv_dir = Path("C:/Users/User/Downloads/python/docking_csv")

# Crear directorios si no existen
results_dir.mkdir(parents=True, exist_ok=True)
csv_dir.mkdir(parents=True, exist_ok=True)

# Crear archivo CSV para almacenar resultados
csv_file = csv_dir / "docking_results.csv"
if not csv_file.exists():
    with open(csv_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Receptor", "Ligand", "Pose", "Affinity (kcal/mol)"])

# Función para ejecutar docking con AutoDock Vina
def run_vina():
    for ligand_file in ligands_dir.glob("*.pdbqt"):
        ligand_name = ligand_file.stem
        output_pose_file = results_dir / f"{ligand_name}_poses.pdbqt"
        output_log_file = results_dir / f"{ligand_name}_log.txt"
        
        command = [
            vina_path,
            "--receptor", str(receptor_file),
            "--ligand", str(ligand_file),
            "--center_x", str(pocket_center[0]),
            "--center_y", str(pocket_center[1]),
            "--center_z", str(pocket_center[2]),
            "--size_x", str(ligand_box[0]),
            "--size_y", str(ligand_box[1]),
            "--size_z", str(ligand_box[2]),
            "--exhaustiveness", "8",
            "--num_modes", "10",
            "--out", str(output_pose_file),
            "--log", str(output_log_file)
        ]
        
        try:
            subprocess.run(command, check=True)
            print(f"Docking exitoso para {ligand_name}, resultados guardados en {output_pose_file}")
            
            # Extraer los resultados del archivo de log
            results = []
            with open(output_log_file, "r") as log_file:
                for line in log_file:
                    if "Affinity" in line:
                        score = line.split()[1]
                        results.append(score)
            
            # Guardar resultados en CSV
            docking_results = [
                {"Receptor": receptor_file.stem, "Ligand": ligand_name, "Pose": i + 1, "Affinity (kcal/mol)": score}
                for i, score in enumerate(results)
            ]
            
            with open(csv_file, mode="a", newline="") as csvfile:
                csv_writer = csv.DictWriter(csvfile, fieldnames=["Receptor", "Ligand", "Pose", "Affinity (kcal/mol)"])
                csv_writer.writerows(docking_results)
            print(f"Resultados guardados: {csv_file}")
        
        except subprocess.CalledProcessError as e:
            print(f"Error en docking de {ligand_name}: {e}")

run_vina()
print("Proceso de docking completado.")


Centro de la caja: [-32.60554171 -21.38570849   1.85266667]
Error en docking de alaproclate: Command '['C:\\Users\\User\\Downloads\\python\\AutodockVina\\vina.exe', '--receptor', 'C:\\Users\\User\\Downloads\\python\\receptores\\5I6X.pdbqt', '--ligand', 'C:\\Users\\User\\Downloads\\python\\ligands_pdbqt\\alaproclate.pdbqt', '--center_x', '-32.605541706085205', '--center_y', '-21.385708491007488', '--center_z', '1.8526666737549629', '--size_x', '5.0', '--size_y', '5.0', '--size_z', '5.0', '--exhaustiveness', '8', '--num_modes', '10', '--out', 'C:\\Users\\User\\Downloads\\python\\docking_results\\alaproclate_poses.pdbqt', '--log', 'C:\\Users\\User\\Downloads\\python\\docking_results\\alaproclate_log.txt']' returned non-zero exit status 1.
Error en docking de amitriptyline: Command '['C:\\Users\\User\\Downloads\\python\\AutodockVina\\vina.exe', '--receptor', 'C:\\Users\\User\\Downloads\\python\\receptores\\5I6X.pdbqt', '--ligand', 'C:\\Users\\User\\Downloads\\python\\ligands_pdbqt\\amitrip